In [7]:
import time
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

# 손 랜드마커와 결과를 관리하는 클래스
class HandLandmarkerAndResult:
    def __init__(self):
        # 결과 저장을 위한 초기화
        self.result = mp.tasks.vision.HandLandmarkerResult
        # 손 랜드마커 초기화
        self.landmarker = mp.tasks.vision.HandLandmarker
        self.create_landmarker()
    
    def create_landmarker(self):
        # 결과 업데이트를 위한 콜백 함수
        def update_result(result: mp.tasks.vision.HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
            self.result = result
            

        # HandLandmarker 옵션 설정
        options = mp.tasks.vision.HandLandmarkerOptions(
            base_options=mp.tasks.BaseOptions(model_asset_path="hand_landmarker.task"),  # 모델 경로 설정
            running_mode=mp.tasks.vision.RunningMode.LIVE_STREAM,  # 실시간 스트림 모드
            num_hands=2,  # 양손 추적
            min_hand_detection_confidence=0.3,  # 손 검출 최소 신뢰도
            min_hand_presence_confidence=0.3,  # 손 존재 최소 신뢰도
            min_tracking_confidence=0.3,  # 추적 최소 신뢰도
            result_callback=update_result  # 결과 콜백 함수 설정
        )
        
        # 손 랜드마커 생성
        self.landmarker = self.landmarker.create_from_options(options)
    
    def detect_async(self, frame):
        # numpy 프레임을 MediaPipe 이미지로 변환
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        # 비동기적으로 랜드마크 검출
        self.landmarker.detect_async(image=mp_image, timestamp_ms=int(time.time() * 1000))

    def close(self):
        # 손 랜드마커 닫기
        self.landmarker.close()





In [8]:
# 이미지에 손 랜드마크를 그리는 함수
def draw_landmarks_on_image(rgb_image, detection_result: mp.tasks.vision.HandLandmarkerResult):
    """이미지에 손 랜드마크를 그립니다."""
    try:
        if not detection_result.hand_landmarks:
            return rgb_image
        
        hand_landmarks_list = detection_result.hand_landmarks
        annotated_image = np.copy(rgb_image)

        for idx, hand_landmarks in enumerate(hand_landmarks_list):
            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend([
                landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
            ])
            mp.solutions.drawing_utils.draw_landmarks(
                annotated_image,
                hand_landmarks_proto,
                mp.solutions.hands.HAND_CONNECTIONS,
                mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                mp.solutions.drawing_styles.get_default_hand_connections_style()
            )

        return annotated_image
    except Exception as e:
        print(f"랜드마크 그리기 오류: {e}")
        return rgb_image


In [21]:
# 손가락이 몇 개 올라와 있는지 세는 함수
def count_fingers_raised_test(rgb_image, detection_result: mp.tasks.vision.HandLandmarkerResult):
    try:
      # 데이터 가져오기
        hand_landmarks_list = detector_result.hand_landmarks 
      # 올린 손가락 수를 계산하는 코드가 여기에 표시됩니다
        numRaised = 0 
      # 올려진 손가락 수를 표시하는 코드가 표시됩니다.
        annotated_image = np.copy(rgb_image) 
        
        return annotated_image
    
    except : 
        return rgb_image 

In [25]:
# 손가락이 몇 개 올라와 있는지 세는 함수
def count_fingers_raised_test(rgb_image, detection_result: mp.tasks.vision.HandLandmarkerResult):
    try:
        # 데이터 가져오기
        hand_landmarks_list = detector_result.hand_landmarks 
        # 각 손가락의 데이터 정보 열기
        print("Hello")
        print(hand_landmarks_list[0][0])
        
            
        numRaised = 0 
        # 올려진 손가락 수를 표시하는 코드가 표시됩니다.
        annotated_image = np.copy(rgb_image) 
        
        return annotated_image
    
    except : 
        return rgb_image 

In [9]:
# 손가락이 몇 개 올라와 있는지 세는 함수
def count_fingers_raised(rgb_image, detection_result: mp.tasks.vision.HandLandmarkerResult):
    """이미지에서 손가락이 몇 개 올라와 있는지 셉니다."""
    try:
        hand_landmarks_list = detection_result.hand_landmarks
        num_raised = 0

        for hand_landmarks in hand_landmarks_list:
            # 각 손가락 끝이 손가락의 다른 부분들보다 높은지 확인
            for i in range(8, 21, 4):
                tip_y = hand_landmarks[i].y
                dip_y = hand_landmarks[i-1].y
                pip_y = hand_landmarks[i-2].y
                mcp_y = hand_landmarks[i-3].y
                if tip_y < min(dip_y, pip_y, mcp_y):
                    num_raised += 1

            # 엄지손가락이 손바닥 방향으로 움직였는지 확인
            tip_x = hand_landmarks[4].x
            dip_x = hand_landmarks[3].x
            pip_x = hand_landmarks[2].x
            mcp_x = hand_landmarks[1].x
            palm_x = hand_landmarks[0].x
            if mcp_x > palm_x:
                if tip_x > max(dip_x, pip_x, mcp_x):
                    num_raised += 1
            else:
                if tip_x < min(dip_x, pip_x, mcp_x):
                    num_raised += 1
        
        # 이미지에 손가락 개수 표시
        annotated_image = np.copy(rgb_image)
        height, width, _ = annotated_image.shape
        text_x = int(hand_landmarks[0].x * width) - 100
        text_y = int(hand_landmarks[0].y * height) + 50
        cv2.putText(annotated_image, f"{num_raised} Fingers Raised", (text_x, text_y),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2, cv2.LINE_4)
        return annotated_image
    except Exception as e:
        print(f"손가락 세기 오류: {e}")
        return rgb_image



In [10]:
# 메인 함수
def main():
    cap = cv2.VideoCapture(0)  # 웹캠 접근

    hand_landmarker = HandLandmarkerAndResult()  # 손 랜드마커 객체 생성

    while True:
        ret, frame = cap.read()  # 프레임 가져오기
        frame = cv2.flip(frame, 1)  # 좌우 반전
        hand_landmarker.detect_async(frame)  # 비동기적으로 손 랜드마크 검출
        frame = draw_landmarks_on_image(frame, hand_landmarker.result)  # 프레임에 랜드마크 그리기
        frame = count_fingers_raised(frame, hand_landmarker.result)  # 손가락 개수 세기
        cv2.imshow('frame', frame)  # 결과 이미지 출력
        if cv2.waitKey(1) == ord('q'):  # 'q' 키를 누르면 종료
            print(hand_landmarker.result)
            break
    
    hand_landmarker.close()  # 손 랜드마커 닫기
    cap.release()  # 웹캠 해제
    cv2.destroyAllWindows()  # 모든 윈도우 닫기

if __name__ == "__main__":
    main()

랜드마크 그리기 오류: type object 'HandLandmarkerResult' has no attribute 'hand_landmarks'
손가락 세기 오류: type object 'HandLandmarkerResult' has no attribute 'hand_landmarks'
랜드마크 그리기 오류: type object 'HandLandmarkerResult' has no attribute 'hand_landmarks'
손가락 세기 오류: type object 'HandLandmarkerResult' has no attribute 'hand_landmarks'
랜드마크 그리기 오류: type object 'HandLandmarkerResult' has no attribute 'hand_landmarks'
손가락 세기 오류: type object 'HandLandmarkerResult' has no attribute 'hand_landmarks'
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류:

손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before assignment
손가락 세기 오류: local variable 'hand_landmarks' referenced before ass

In [11]:
import cv2
import mediapipe as mp
import pyautogui as pg

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands()

cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH,1080)
width = cam.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cam.get(cv2.CAP_PROP_FRAME_HEIGHT)

com_width, com_height = pg.size()


compare = [(17,4,2),(0,8,5),(0,12,9),(0,16,13),(0,20,17)]

while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame, 1) 
    if not ret: continue
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(image)
    if result.multi_handedness:
        
        for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
            a = hand_landmarks.landmark

            finger_point = (int(a[8].x*com_width),int(a[8].y*com_height))
            pg.moveTo(finger_point[0],finger_point[1])

            label = handedness.classification[0].label
            coord = (int(a[0].x * width), int(a[0].y * height))
            cv2.putText(frame, label, coord, cv2.FONT_HERSHEY_SIMPLEX, 2, (25,16,100), 5)
            mp_drawing.draw_landmarks(frame, hand_landmarks, 
                                      mp_hands.HAND_CONNECTIONS)
            folding = ""
            for base,tip,mcp in compare:
                base_tip = (a[base].x-a[tip].x)**2 + (a[base].y-a[tip].y)**2
                base_mcp = (a[base].x-a[mcp].x)**2 + (a[base].y-a[mcp].y)**2
                # "01"[base_tip < base_mcp]
                folding += str(int(base_tip < base_mcp))
            coord2= (coord[0],coord[1]+30)
            cv2.putText(frame, folding, coord2,cv2.FONT_HERSHEY_SIMPLEX, 2, (255,36,125), 5)

    cv2.imshow("cv2 test", frame)

    if cv2.waitKey(50) == 27: break

cam.release()
cv2.destroyAllWindows()